In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, clear_output
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
dataset= pd.read_csv("stroke prediction.csv")
dataset.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [3]:
dataset.drop(columns=["id","Residence_type"],inplace=True)

In [4]:
dataset["gender"]=dataset["gender"].astype('category')
dataset["ever_married"]=dataset["ever_married"].astype('category')
# dataset["Residence_type"]=dataset["Residence_type"].astype('category')
dataset["work_type"]=dataset["work_type"].astype('category')
dataset["smoking_status"]=dataset.smoking_status.astype("category")

In [5]:
dataset_imputed=dataset.copy()

In [6]:
dataset_imputed.dtypes

gender               category
age                   float64
hypertension            int64
heart_disease           int64
ever_married         category
work_type            category
avg_glucose_level     float64
bmi                   float64
smoking_status       category
stroke                  int64
dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

dataset_imputed.gender=encoder.fit_transform(dataset_imputed.gender)

dataset_imputed.ever_married=encoder.fit_transform(dataset_imputed.ever_married)

dataset_imputed.work_type=encoder.fit_transform(dataset_imputed.work_type)

# dataset_imputed.Residence_type=encoder.fit_transform(dataset_imputed.Residence_type)

dataset_imputed.smoking_status=dataset_imputed.smoking_status.map({'never smoked':0,'formerly smoked':1,'smokes':2})

In [8]:
X=dataset_imputed.drop(columns="stroke",axis=1)
y=dataset_imputed["stroke"]

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.33,random_state=88,stratify=y)

In [9]:
from sklearn.impute import KNNImputer

imputer=KNNImputer()

X_trimputed=pd.DataFrame(imputer.fit_transform(X_train),columns=X_train.columns)

X_teimputed=pd.DataFrame(imputer.transform(X_test),columns=X_test.columns)

In [10]:
X_trimputed["smoking_status"]=X_trimputed["smoking_status"].apply(round)
X_trimputed["smoking_status"].value_counts()

# X_trimputed["smoking_status"]=X_trimputed["smoking_status"].astype('category')
X_trimputed["work_type"]=X_trimputed["work_type"].apply(int)
# X_trimputed["work_type"]=X_trimputed["work_type"].astype('category')

In [11]:
X_teimputed["smoking_status"]=X_teimputed["smoking_status"].apply(round)
X_teimputed["smoking_status"].value_counts()

# X_teimputed["smoking_status"]=X_teimputed["smoking_status"].astype('category')
X_teimputed["work_type"]=X_teimputed["work_type"].apply(int)
# X_teimputed["work_type"]=X_teimputed["work_type"].astype('category')

In [12]:
# X_trimputed["gender"]=X_trimputed["gender"].astype("category")
X_trimputed[["hypertension","heart_disease","ever_married"]]=X_trimputed[["hypertension","heart_disease","ever_married"]].astype("int")

# X_teimputed["gender"]=X_teimputed["gender"].astype("category")
X_teimputed[["hypertension","heart_disease","ever_married"]]=X_teimputed[["hypertension","heart_disease","ever_married"]].astype("int")

In [13]:
X_trimputed=pd.get_dummies(X_trimputed,drop_first=True)
X_teimputed=pd.get_dummies(X_teimputed,drop_first=True)

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

scaler.fit(X_trimputed)

X_trimputed=pd.DataFrame(scaler.transform(X_trimputed),columns=X_trimputed.columns)

X_teimputed=pd.DataFrame(scaler.transform(X_teimputed),columns=X_teimputed.columns)

In [15]:
from sklearn.metrics import classification_report,roc_auc_score,f1_score

In [16]:
X_trimputed

,gender,age,hypertension,heart_disease,ever_married,work_type,avg_glucose_level,bmi,smoking_status
0,0.0,0.865723,0.0,0.0,1.0,0.50,0.187511,0.219114,1.0
1,0.0,0.987793,0.0,0.0,1.0,0.50,0.188400,0.196970,0.0
2,0.0,0.609375,0.0,0.0,1.0,0.50,0.061769,0.136364,0.5
3,0.5,0.133301,0.0,0.0,0.0,1.00,0.125021,0.127040,0.0
4,0.5,0.731445,0.0,0.0,1.0,0.00,0.089731,0.219114,1.0
...,...,...,...,...,...,...,...,...,...
29073,0.0,0.890137,0.0,0.0,1.0,0.00,0.065582,0.033800,0.0
29074,0.0,0.523926,0.0,0.0,0.0,0.00,0.168658,0.226107,0.0
29075,0.0,0.829102,1.0,0.0,1.0,0.75,0.640061,0.191142,0.0
29076,0.0,0.279785,0.0,0.0,0.0,0.50,0.184926,0.209790,0.5


# RandomForest

In [18]:
#creating an Gradient boosting instance
clf = RandomForestClassifier(random_state=40)

In [19]:
#training the model
clf.fit(X_trimputed,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=40, verbose=0,
                       warm_start=False)

In [20]:
y_train_pred=clf.predict(X_trimputed)
y_test_pred=clf.predict(X_teimputed)

In [21]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28553
           1       1.00      1.00      1.00       525

    accuracy                           1.00     29078
   macro avg       1.00      1.00      1.00     29078
weighted avg       1.00      1.00      1.00     29078



In [22]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     14064
           1       0.00      0.00      0.00       258

    accuracy                           0.98     14322
   macro avg       0.49      0.50      0.50     14322
weighted avg       0.96      0.98      0.97     14322



In [24]:
y_train_prob=clf.predict_proba(X_trimputed)[:,1]
y_test_prob=clf.predict_proba(X_teimputed)[:,1]

In [25]:
roc_auc_score(y_test,y_test_prob)

0.7580880261660978

## Grid Search

In [37]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [38]:
parameters={'class_weight':[None,'balanced']
            ,'n_estimators':np.arange(100,1000,100),
            'max_depth':np.arange(2,20,1)}

In [39]:
search=RandomizedSearchCV(RandomForestClassifier(random_state=40,n_jobs=-1),parameters,n_jobs=-1,scoring='roc_auc',cv=5,verbose=2,n_iter=100)

In [ ]:
search.fit(X_trimputed,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 12.8min


In [73]:
search.best_score_

0.8496261005653265

In [74]:
search.best_params_

{'subsample': 1,
 'max_depth': 2,
 'learning_rate': 0.8999999999999999,
 'gamma': 2.2,
 'base_score': 0.2}

In [60]:
#creating an Gradient boosting instance
clf = XGBClassifier(random_state=96, max_depth=9, learning_rate=2.3, gamma=2.2, base_score=0.3, n_estimators=200, subsample=0.7)

In [61]:
#training the model
clf.fit(X_trimputed,y_train)

XGBClassifier(base_score=0.3, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=2.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=2.3, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=96, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [62]:
y_train_pred=clf.predict(X_trimputed)
y_test_pred=clf.predict(X_teimputed)

In [63]:
y_train_prob=clf.predict_proba(X_trimputed)[:,1]
y_test_prob=clf.predict_proba(X_teimputed)[:,1]

In [64]:
print(classification_report(y_train,y_train_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     28553
           1       0.25      0.28      0.26       525

    accuracy                           0.97     29078
   macro avg       0.62      0.63      0.62     29078
weighted avg       0.97      0.97      0.97     29078



In [65]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     14064
           1       0.07      0.09      0.08       258

    accuracy                           0.96     14322
   macro avg       0.53      0.53      0.53     14322
weighted avg       0.97      0.96      0.96     14322



In [66]:
roc_auc_score(y_test,y_test_prob)

0.5967858174370101

## Grid Search (F1 score)

In [67]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [68]:
parameters={'subsample':[0.7,0.8,0.9,1],
            'base_score':[0.2,0.3,0.4,0.5,0.6],
            'gamma':np.arange(0,2.5,0.1),
            'learning_rate':np.arange(0.5,2.5,0.1),
            'max_depth':[2,3,4,5,6,7,8,9,10]}

In [71]:
search=RandomizedSearchCV(XGBClassifier(random_state=40),parameters,scoring='f1',cv=5,n_jobs=-1,verbose=2,n_iter=500)

In [72]:
search.fit(X_trimputed,y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1961 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 10.6min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n...
                                        'gamma': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.

In [73]:
search.best_score_

0.8496261005653265

In [74]:
search.best_params_

{'subsample': 1,
 'max_depth': 2,
 'learning_rate': 0.8999999999999999,
 'gamma': 2.2,
 'base_score': 0.2}